In [1]:
from learning_lib.nn.ffnn import FFNN

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import plotly.offline as plotly
import plotly.graph_objs as go

from PIL import Image

/Users/frankwang/anaconda3/envs/learning_lib/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/frankwang/anaconda3/envs/learning_lib/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
plotly.init_notebook_mode(connected=True)

# Data

In [3]:
dataset = input_data.read_data_sets("MNIST_data/", one_hot=True)

train_data = dataset.train.next_batch(1000000)
train_in = train_data[0] - train_data[0].mean()
train_labels = train_data[1]

test_data = dataset.test.next_batch(1000000)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# MSE Loss

In [ ]:
network = FFNN([784, 2048, 2048, 10])

In [109]:
%%time
network.train(train_in, train_labels, epochs=81400, batch_size=50, lc_interval=200)

Reached epoch 79200
Reached epoch 79400
Reached epoch 79600
Reached epoch 79800
Reached epoch 80000
Reached epoch 80200
Reached epoch 80400
Reached epoch 80600
Reached epoch 80800
Reached epoch 81000
Reached epoch 81200
CPU times: user 32min 1s, sys: 6min 17s, total: 38min 19s
Wall time: 13min 36s


In [110]:
train_pred = network.evaluate(train_in)

In [111]:
np.sum(train_pred.argmax(axis=1) == train_labels.argmax(axis=1)) / train_in.shape[0]

0.80050909090909095

In [ ]:
lc = pd.DataFrame(network.learning_curve)
plotly.iplot([go.Scatter(
    x=lc[0],
    y=lc[1],
    mode='lines'
)])

# Cross Entropy Loss

In [29]:
lc = [
    784,
    {
        'n_nodes': 2048, 'activation': tf.nn.tanh, 'init_weight_lower': 0, 'init_weight_upper': 1,
        'init_bias_lower': 0, 'init_bias_upper': 1
    },
    {
        'n_nodes': 10, 'activation': tf.identity, 'init_weight_lower': 0, 'init_weight_upper': 1,
        'init_bias_lower': 0, 'init_bias_upper': 1
    }
]

In [30]:
network = FFNN(lc, post_proc_function=tf.nn.softmax)

In [31]:
def cross_entropy_with_softmax(model_output, true_output):
    return tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=true_output, logits=model_output))

In [37]:
%%time
network.train(
    train_in,
    train_labels,
    epochs=1100,
    batch_size=100,
    report_interval=200,
    loss_func=cross_entropy_with_softmax,
    optimizer=tf.train.GradientDescentOptimizer(0.1)
)

Reached epoch 1200
Reached epoch 1400
Reached epoch 1600
Reached epoch 1800
Reached epoch 2000
CPU times: user 7min 5s, sys: 2min 33s, total: 9min 38s
Wall time: 5min 34s


In [34]:
train_pred = network.evaluate(train_in)

In [35]:
np.sum(train_pred.argmax(axis=1) == train_labels.argmax(axis=1)) / train_in.shape[0]

0.8635272727272727

In [36]:
lc = pd.DataFrame(network.learning_curve)
plotly.iplot([go.Scatter(
    x=lc[0][1:],
    y=lc[1][1:],
    mode='lines'
)])